In [1]:
%%time

import tkinter as tk
from tkinter import filedialog 
import tkinter.messagebox as tm
from tkinter import *
import spacy
import nltk
from nltk.corpus import wordnet as wn
from spacy.lang.en import English
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.summarization import summarize
from gensim.summarization import keywords
from gensim import corpora
import pickle
import gensim
import pyLDAvis.gensim
import re
import speech_recognition as sr
import subprocess
import uuid
import os
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn import preprocessing
import python_speech_features as mfcc
from sklearn.mixture import GMM
from punctuator import Punctuator
p = Punctuator('Demo-Europarl-EN.pcl')

datafile=""
topicfile=""
speakerfile=""
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

en_stop = set(nltk.corpus.stopwords.words('english'))


def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def SpeakerUpload(event=None):
    global speakerfile

    def calculate_delta(array):
        """Calculate and returns the delta of given feature vector matrix"""

        rows,cols = array.shape
        deltas = np.zeros((rows,20))
        N = 2
        for i in range(rows):
            index = []
            j = 1
            while j <= N:
                if i-j < 0:
                    first = 0
                else:
                    first = i-j
                if i+j > rows -1:
                    second = rows -1
                else:
                    second = i+j
                index.append((second,first))
                j+=1
            deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
        return deltas

    def extract_features(audio,rate):
        """extract 20 dim mfcc features from an audio, performs CMS and combines 
        delta to make it 40 dim feature vector"""    
    
        mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    
        mfcc_feat = preprocessing.scale(mfcc_feat)
        delta = calculate_delta(mfcc_feat)
        combined = np.hstack((mfcc_feat,delta)) 
        return combined



    pathspeaker=filedialog.askopenfilename()
    print('Selected:',pathspeaker)
#path to training data
    source   = "development_set\\"   

#path where training speakers will be saved
    dest = "speaker_models\\"

    train_file = "development_set_enroll.txt"        


    file_paths = open(train_file,'r')

    count = 1

# Extracting features for each speaker (5 files per speakers)
    features = np.asarray(())
    for path in file_paths:    
        path = path.strip()   
    
    # read the audio
        sr,audio = read(source + path)
    
    # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(audio,sr)
    
        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
    # when features of 5 files of speaker are concatenated, then do model training
        if count == 5:    
            gmm = GMM(n_components = 16, n_iter = 200, covariance_type='diag',n_init = 3)
            gmm.fit(features)
        
        # dumping the trained gaussian model
            picklefile = path.split("-")[0]+".gmm"
            cPickle.dump(gmm,open(dest + picklefile,'wb'))
  
            features = np.asarray(())
            count = 0
        count = count + 1
    
    #test_gender.py



#path to training data
    source   = "development_set\\"   

    modelpath = "speaker_models\\"


    gmm_files = [os.path.join(modelpath,fname) for fname in 
                  os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the Gaussian gender Models
    models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
    speakers   = [fname.split("\\")[-1].split(".gmm")[0] for fname 
                  in gmm_files]

# Read the test directory and get the list of test audio files 
    #pathspeaker=filedialog.askopenfilename()
    #print('Selected:',pathspeaker)   
    sr,audio = read(pathspeaker)
    vector   = extract_features(audio,sr)
    log_likelihood = np.zeros(len(models)) 
    
    for i in range(len(models)):
        gmm    = models[i]         #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    
    winner = np.argmax(log_likelihood)
    speakerfile=speakers[winner]
    print("\tdetected as - ",speakers[winner])
    time.sleep(1.0)

def AudioUpload(event=None):
    global datafile
    global topicfile
    global datafile1
    
    filename = filedialog.askopenfilename()
    print('Selected:', filename)
    filename1="audio.wav"
    command = [r"C:\Users\Rohit\Downloads\ffmpeg-20200403-52523b6-win64-static\ffmpeg-20200403-52523b6-win64-static\bin\ffmpeg.exe","-y","-i", filename, filename1] #-y is used so that you can overwrite the audio file
    subprocess.run(command,shell=True)
    r = sr.Recognizer()
    
    with sr.AudioFile(filename1) as source:
        r.adjust_for_ambient_noise(source)
        audio = r.record(source)
    try:
        datafile1=r.recognize_sphinx(audio)
        print(type(datafile1))
        print("Sphinx thinks you said " + datafile1)
    except sr.UnknownValueError:
        print("Sphinx could not understand audio")
    except sr.RequestError as e:
        print("Sphinx error; {0}".format(e))
    
    unique_filename = str(uuid.uuid4())
    print(unique_filename)
    f = open(unique_filename+".txt", "w+",encoding='utf-8')
    print(type(datafile1))
    v=f.write(datafile1)
    f.close()
    
    text_data = []
    summ=[]
    f = open(unique_filename+".txt", "r",encoding='utf-8')
    for line in f:
        m=p.punctuate(line)
        summ.append(summarize(m,ratio=0.05))
        tokens = prepare_text_for_lda(line)
        if True:
            #print(tokens)
            text_data.append(tokens) 
    datafile=summ
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('dictionary.gensim')
    

    NUM_TOPICS = 2
    dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
    corpus = pickle.load(open('corpus.pkl', 'rb'))
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save('model3.gensim')
    topicfile = ldamodel.print_topics(num_words=5)
    print(topicfile)

    lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')
    
    # to extract words from string 
    # using regex( findall() ) 
  
    test_string = topicfile[0][1]

    # using regex( findall() ) 
    # to extract words from string 
    res = re.findall(r'\w+', test_string) 

    # Python program to Remove all digits from a list of string 
    from string import digits 

    def remove(listr): 
        listr = [''.join(x for x in i if x.isalpha()) for i in listr] 
        return listr

    # Driver code 

    listr = res
    v=remove(listr)
    #print(v)

    # Function to convert 
    def listToString(s): 
        str1 = "" 
    
    # traverse in the string 
        for ele in s: 
            str1 += ele+' '  
        return str1 

    # Driver code	 
    s = v
    topicfile=listToString(s)
    print(topicfile)
    
    print(datafile[0])
    #lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
    #pyLDAvis.display(lda_display)

def UploadAction(event=None):
    global datafile
    global topicfile
    filename = filedialog.askopenfilename()
    print('Selected:', filename) 
    
    text_data = []
    summ=[]
    with open(filename, encoding="utf8") as f:
        for line in f:
            print("Hello",f)
            summ.append(summarize(line,ratio=0.05))
            tokens = prepare_text_for_lda(line)
            if True:
                #print(tokens)
                text_data.append(tokens)
    datafile=summ         
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('dictionary.gensim')
    

    NUM_TOPICS = 2
    dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
    corpus = pickle.load(open('corpus.pkl', 'rb'))
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save('model3.gensim')
    topicfile = ldamodel.print_topics(num_words=5)
    print(topicfile)

    lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')
    
    # to extract words from string using regex( findall() )
    test_string = topicfile[0][1]

    # using regex( findall() ) to extract words from string 
    res = re.findall(r'\w+', test_string) 

 
    # digits from a list of string 
    from string import digits 

    def remove(listr): 
        listr = [''.join(x for x in i if x.isalpha()) for i in listr] 
        return listr

    # Driver code 

    listr = res
    v=remove(listr)
    #print(v)

    # Function to convert 
    def listToString(s): 

    # initialize an empty string 
        str1 = "" 
    
    # traverse in the string 
        for ele in s: 
            str1 += ele+' ' 
        # return string 
        return str1 

    # Driver code	 
    s = v
    topicfile=listToString(s) 
    print(topicfile)
    
    print(datafile[0])
    #lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
    #pyLDAvis.display(lda_display)
    
def Display_function():
    global topicfile
    print(topicfile)
    
    textBox2.delete('1.0', END)
    textBox2.insert(tk.END, topicfile)

def Summary_function():
    global datafile
    print(datafile)
    
    textBox1.delete('1.0', END)
    textBox1.insert(tk.END, datafile[0])

def SpeakerDisplay():
    global datafile
    print(datafile)
    
    textBox3.delete('1.0', END)
    textBox3.insert(tk.END, speakerfile)

window = tk.Tk()
#window.geometry("200x200+30+30")
window.state('zoomed')
window.title("MINUTES of MEETING PROTOTYPE")
button1 = tk.Button(window, text='Upload Text File', command=UploadAction)
button4 = tk.Button(window, text='Upload Audio/Video\nFile', command=AudioUpload)
button5 = tk.Button(window, text='Upload Speaker', command=SpeakerUpload)
button2 = tk.Button(window, text='Keywords', command=Display_function)
button3 = tk.Button(window, text='Summarize', command=Summary_function)
button6 = tk.Button(window, text='Speaker', command=SpeakerDisplay)

#keywords
display = tk.Label(window)
textBox2=Text(window, height=10, width=30)
display.pack()
o = tk.Label(window, text="Relevant Keywords")
o.pack()
o.place(x = 360, y = 40, width=120, height=50)
textBox2.pack()
textBox2.place(x = 300, y = 90)

#summary
display = tk.Label(window)
textBox1=Text(window, height=10, width=80)
display.pack()
o = tk.Label(window, text="Summary of the topic")
o.pack()
o.place(x = 850, y = 40, width=120, height=50)
textBox1.pack()
textBox1.place(x = 620, y= 90)

#speaker
display = tk.Label(window)
textBox3=Text(window, height=8, width=60)
display.pack()
o = tk.Label(window, text="Speaker Name")
o.pack()
o.place(x = 880, y =400, width=120, height=50)
textBox3.pack()
textBox3.place(x = 700, y= 450)

#upload text file
button1.pack()
button1.place(x = 70, y = 100, width=120, height=50)
#keywords
button2.pack()
button2.place(x = 350, y = 280, width=120, height=50)
#summarize
button3.pack()
button3.place(x = 860, y = 280, width=120, height=50)
#upload Audio/Video
button4.pack()
button4.place(x= 70, y =180,width=120, height=80)
o = tk.Label(window, text="Processing takes\n2-3 minutes based\n on Audio file size")
o.pack()
o.place(x = 70, y = 270, width=120, height=50)
#Upload Speaker file
button5.pack()
button5.place(x= 450, y =450,width=120, height=50)
o = tk.Label(window, text="Processing takes\n15-20 seconds")
o.pack()
o.place(x = 450, y = 500, width=120, height=50)
#speaker
button6.pack()
button6.place(x= 880, y =600,width=120, height=50)

window.mainloop()

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Selected: C:/Users/Rohit/testspeaker.wav
	detected as -  anthonyschaller

Wall time: 11min
